##2025년 금융데이터 분석사   09.26

### 문제_01 데이터 수집 및 시각화

## 문제 개요

한국은행 ECOS API에서 재무상태표(stat_code=501Y001)의 전산업(ZZZ00) 항목을 내려받아, 원시 데이터에 한 해(예: 2009)에 여러 세부항목(여러 행)이 존재하는 경우를 고려하여 전처리(연도별 집계 포함)하고 지정된 시각화를 생성하시오. 이후 연간 시계열을 바탕으로 간단한 수치분석을 수행하시오.

##1) 데이터 수집 정보

- 통계표 코드 (stat_code): 501Y001
- 주기 (frequency): A (연간, YYYY)
- 세부항목 (item_code1): ZZZ00 (전산업 / All industries)
- 분석 기간: 2009 (2009년) 부터 2023 (2023년) 까지

https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/9999/501Y001/A/2009/2023/ZZZ00


## 2) 전처리 요구 사항 (핵심: 연도별 집계 포함)


아래 항목은 반드시 지켜야 합니다. 구현 방법(사용 함수)은 자유이나 결과(형식)는 채점 기준에 맞아야 합니다.

### 기본 변환

수집된 데이터의  TIME 값을 year(정수)와 date(YYYY-01-01 형식의 datetime)로 변환하시오.

date를 인덱스로 설정하고 오름차순 정렬하시오.

(확인 코드) 원시 DataFrame(집계 전)의 상위 10행을 출력하여 동일 연도에 여러 행이 존재하는지 보여주시오.

### 값 컬럼 식별 및 숫자 변환

숫자 컬럼(예: DATA_VALUE)을 찾아 내부 쉼표(,) 제거 후 pd.to_numeric(..., errors='coerce')로 변환. 변환 불가 항목은 NaN 처리.

### 연도별 집계 (필수)

집계 기준: 동일 연도의 여러 행을 연도 단위로 합계(sum) 하여 연간값을 만든다.

구현 예시(개념): annual = df.groupby('year')['value'].sum() → 연간 DataFrame 생성.

집계 후에는 연간 DataFrame(연도당 1행)이 되어야 하며, 이 연간 시계열을 기준으로 다음 단계 진행.

### 결측치 처리

집계된 연간 시계열(value)에서 결측이 발생한 경우 선택된 간단한 처리 방식: interpolate() 또는 ffill().bfill() 중 하나를 사용하시오.

시험 지침(권고): 이번 문제에서는 interpolate(method='linear', limit_direction='both')를 사용하여 연간 결측을 보간하시오. (이유: 연간 합계의 연속성을 합리적으로 보장함)

### [전처리 작성 중 새롭게 추가될 컬럼 정보]

ma_3: 연간 value의 3년 이동평균 (rolling(window=3, min_periods=1).mean())

pct_yoy: 전년대비(%) = pct_change(periods=1) * 100

cumsum: 연간 value의 누적합 (cumsum)


## 3) 시각화 요구 사항 (연간 집계 결과 기반)


3-1) 그래프 A — Scatter + Trendline

내용: 집계된 연(年)별 value에 대해 산점도(scatter)를 그리고, 단순 선형회귀(trendline)를 그려 기울기(또는 slope) 값을 범례 또는 텍스트로 표기하시오.

시각 요소:

제목: "Scatter with Trendline"

x축: "Year"

y축: "Value (unit as provided)"

범례: 산점도와 회귀선 구분. 회귀선에는 slope=... 형식으로 기울기 표기(소수점 2자리).

설계 팁: np.polyfit(x, y, 1) 또는 scipy.stats.linregress를 이용하면 기울기와 절편을 얻을 수 있음(학생 선택).

3-2) 그래프 B — Cumulative Area

내용: 집계된 연간 value의 누적합(cumsum)을 영역 그래프(fill_between)으로 표시. 추세와 누적총량 증감을 시각화.

시각 요소:

제목: "Cumulative Sum (Area) over Years"

x축: "Year"

y축: "Cumulative Value"

범례: Cumulative Sum 표기, 마커로 연도 포인트를 표시(선 + 마커).

## 제출 파일
1) 전처리된 Dataframe 저장
   -- 최종 전처리된 DataFrame을 금융분석가_문제2.csv 이름으로 저장하시오 (인코딩 utf-8-sig 권장)

2) 작성된 금융분석가_문제2.ipynb

2개 파일을 로컬 PC로 다운로드하여 제출합니다.

## [참고]

- 년도별 집계
<img src="https://imgur.com/rEcf87G.png" width="80%">

- 출력될 결과값

<img src="https://imgur.com/8zrJmaZ.png" width="60%">

<img src="https://imgur.com/YfgIjta.png" width="80%">

<img src="https://imgur.com/Ol5WzAg.png" width="80%">



In [ ]:
# Problem 2 — Answer cell (one cell). Replace api_key and run.
import requests, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,5)
sns.set_style('whitegrid')

